In [4]:
!conda install -c conda-forge geocoder --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                   /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|3_73_pypy|4_73_pypy|2_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your py

In [5]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [6]:
! pip install folium==0.5.0

## Import Modules

In [7]:
import folium
from geopy.geocoders import Nominatim
import json
import pandas as pd
from pandas import json_normalize 
import numpy as np
import requests

## Import Data Set

In [8]:
postal = pd.read_csv('https://raw.githubusercontent.com/carmst28/Coursera-Capstone/main/QLD%20Postcodes-1.csv')
postal.head()

,Post Code,Borough,Neighbourhood,Longitude,Latitude
0,4000,Brisbane Inner City,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",153.022126,-27.468391
1,4001,Brisbane Inner City,"BRISBANE, CENTRAL PLAZA, RIVERSIDE CENTRE, WAT...",153.030681,-27.603479
2,4002,Brisbane Inner City,"BRISBANE ALBERT STREET BC, WINTERGARDEN",153.027161,-27.603479
3,4003,Brisbane Inner City,GEORGE STREET,153.025252,-27.472640
4,4004,Brisbane Inner City,SPRING HILL,153.025016,-27.460912


In [9]:
merged = postal

In [10]:
merged.dropna(inplace=True)

## Shape of Dataframe - determines number of Post Codes

In [11]:
merged.shape

(75, 5)

In [12]:
address = 'Brisbane, Australia'

geolocator = Nominatim(user_agent="brisbane_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brisbane are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brisbane are -27.4689682, 153.0234991.


## Generate a Map of Brisbane with blue pins showing each Post Code

In [13]:
# create map of Brisbane using latitude and longitude values
map_Brisbane = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged['Latitude'], merged['Longitude'], merged['Borough'], merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186ca',
        fill_opacity=0.7,
        ).add_to(map_Brisbane)  
    
map_Brisbane

## Input FourSquare API credentials

In [14]:
CLIENT_ID = 'EB44OMQXOD2YURAIDBSYDSJTTD3CI2OVV4YNZBGYIWQYZ5GX'
CLIENT_SECRET = 'FQ40OQLKHBDOWMIG3Z0A00QNDE2VJHP3ZCKLKBOHOLDOUJ21'
VERSION = '20201224'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EB44OMQXOD2YURAIDBSYDSJTTD3CI2OVV4YNZBGYIWQYZ5GX
CLIENT_SECRET:FQ40OQLKHBDOWMIG3Z0A00QNDE2VJHP3ZCKLKBOHOLDOUJ21


In [15]:
neighborhood_latitude = merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = merged.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = merged.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISBANE GPO, PETRIE TERRACE, SPRING HILL are -27.468390999999997, 153.02212600000001.


## Set up Get Request to API

In [16]:
LIMIT = 100
radius = 250
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EB44OMQXOD2YURAIDBSYDSJTTD3CI2OVV4YNZBGYIWQYZ5GX&client_secret=FQ40OQLKHBDOWMIG3Z0A00QNDE2VJHP3ZCKLKBOHOLDOUJ21&v=20201224&ll=-27.468390999999997,153.02212600000001&radius=250&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fef8451c94739757d9ce52f'},
 'response': {'headerLocation': 'Brisbane City',
  'headerFullLocation': 'Brisbane City, Brisbane',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': -27.466140997749996,
    'lng': 153.0246571492444},
   'sw': {'lat': -27.470641002249998, 'lng': 153.01959485075562}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a68151de65d0c5c76a4c425',
       'name': 'W Brisbane',
       'location': {'address': '81 North Quay',
        'lat': -27.47012,
        'lng': 153.02197,
        'labeledLatLngs': [{'label': 'display',
          'lat': -27.47012,
          'lng': 153.02197}],
        'distance': 193,
        'postalCode': '4000',
        'cc': 'AU',
       

## function that extracts the category of the venue

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Filter Results into a new Dataframe

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,W Brisbane,Hotel,-27.470120,153.021970
1,Frankie & George,Café,-27.468974,153.021558
2,Museum of Brisbane,History Museum,-27.469028,153.023837
3,Three Blue Ducks,Breakfast Spot,-27.470120,153.021970
4,Chocolate Soldier,Café,-27.469858,153.022752


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
Brisbane_venues = getNearbyVenues(names=merged['Neighbourhood'],
                                   latitudes=merged['Latitude'],
                                   longitudes=merged['Longitude']
                                  )

BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISBANE GPO, PETRIE TERRACE, SPRING HILL
BRISBANE, CENTRAL PLAZA, RIVERSIDE CENTRE, WATERFRONT PLACE
BRISBANE ALBERT STREET BC, WINTERGARDEN
GEORGE STREET
SPRING HILL
NEW FARM, TENERIFFE
BOWEN BRIDGE, BOWEN HILLS, BRISBANE EXHIBITION, FORTITUDE VALLEY, HERSTON, MAYNE, NEWSTEAD
ASCOT, BRISBANE AIRPORT, DOOMBEN, HAMILTON, HAMILTON CENTRAL, WHINSTANES
BULWER ISLAND, MEEANDAH, MYRTLETOWN, PINKENBA
EAGLE FARM, EAGLE FARM BC
ALBION, ALBION BC, BREAKFAST CREEK
CLAYFIELD, EAGLE JUNCTION, HENDRA
NUNDAH, TOOMBUL, WAVELL HEIGHTS, WAVELL HEIGHTS NORTH
NORTHGATE, NORTHGATE MC
BANYO. NUDGEE, NUDGEE BEACH, VIRGINIA, VIRGINIA BC
BRACKEN RIDGE, BRIGHTON, BRIGHTON NATHAN STREET, DEAGON, SANDGATE, SANDGATE DC, SHORNCLIFFE
FITZGIBBON, TAIGUM
BULWER, CAPE MORETON, COWAN COWAN, KOORINGAL, MORETON ISLAND, TANGALOOMA
ROYAL BRISBANE HOSPITAL
EILDON HILL, KALINGA, LUTWYCHE, WINDSOR, WOOLOOWIN
GLEN KEDRON, GORDON PARK, KEDRON
CHERMSIDE, CHERMSIDE BC, CHERMSIDE CENTRE, CH

## Display new Dataframe with venues

In [24]:
print(Brisbane_venues.shape)
Brisbane_venues.head()

(553, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",-27.468391,153.022126,W Brisbane,-27.470120,153.021970,Hotel
1,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",-27.468391,153.022126,Frankie & George,-27.468974,153.021558,Café
2,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",-27.468391,153.022126,Museum of Brisbane,-27.469028,153.023837,History Museum
3,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",-27.468391,153.022126,Three Blue Ducks,-27.470120,153.021970,Breakfast Spot
4,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",-27.468391,153.022126,Cartel Coffee,-27.468448,153.024845,Coffee Shop


## Display Grouped Dataframe

In [25]:
Brisbane_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"ALBION, ALBION BC, BREAKFAST CREEK",14,14,14,14,14,14
"ALDERLEY, ENOGGERA, GAYTHORNE, GRANGE, NEWMARKET, THE GRANGE, WILSTON",6,6,6,6,6,6
"ALGESTER, PARKINSON",1,1,1,1,1,1
"ALTANDI, BANOON, MACGREGOR, ROBERTSON, SUNNYBANK, SUNNYBANK HILLS, SUNNYBANK SOUTH",5,5,5,5,5,5
"ANNERLEY, FAIRFIELD, FAIRFIELD GARDENS, THOMPSON ESTATE",15,15,15,15,15,15
...,...,...,...,...,...,...
SPRING HILL,23,23,23,23,23,23
THE GAP,4,4,4,4,4,4
THORNESIDE,4,4,4,4,4,4


## Encode Categorical data with "one-hot-encoding"

In [26]:
Brisbane_onehot = pd.get_dummies(Brisbane_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brisbane_onehot['Neighbourhood'] = Brisbane_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Brisbane_onehot.columns[-1]] + list(Brisbane_onehot.columns[:-1])
Brisbane_onehot = Brisbane_onehot[fixed_columns]

Brisbane_onehot.head()

,Neighbourhood,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,...,Tea Room,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"BRISBANE ADELAIDE STREET, BRISBANE CITY, BRISB...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
Brisbane_onehot.shape

(553, 154)

In [28]:
Brisbane_grouped = Brisbane_onehot.groupby('Neighbourhood').mean().reset_index()
Brisbane_grouped.head()

,Neighbourhood,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,...,Tea Room,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,"ALBION, ALBION BC, BREAKFAST CREEK",0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,...,0.0,0.071429,0.000000,0.0,0.000000,0.071429,0.0,0.0,0.0,0.0
1,"ALDERLEY, ENOGGERA, GAYTHORNE, GRANGE, NEWMARK...",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.166667,0.166667,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0
2,"ALGESTER, PARKINSON",0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,"ALTANDI, BANOON, MACGREGOR, ROBERTSON, SUNNYBA...",0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.0,0.0,0.0
4,"ANNERLEY, FAIRFIELD, FAIRFIELD GARDENS, THOMPS...",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.066667,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


## show potential venue categories

In [29]:
Brisbane_grouped.columns.tolist()

['Neighbourhood',
 'Airport',
 'Airport Lounge',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Asian Restaurant',
 'Athletics & Sports',
 'Australian Restaurant',
 'Automotive Shop',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bar',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Board Shop',
 'Boat Launch',
 'Bookstore',
 'Botanical Garden',
 'Boutique',
 'Bowling Alley',
 'Bowling Green',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Cafeteria',
 'Café',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Comedy Club',
 'Comic Shop',
 'Concert Hall',
 'Construction & Landscaping',
 'Convenience Store',
 'Cricket Ground',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Discount Store',
 'Dumpling Restaurant',
 'Electronics Store',
 'Farmers Market',
 'Fast Food Restaurant',
 'Fish & Chips S

In [30]:
Brisbane_grouped.shape

(65, 154)

## create function to return top 10 most common venues

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood',]
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Brisbane_grouped['Neighbourhood']

for ind in np.arange(Brisbane_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brisbane_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"ALBION, ALBION BC, BREAKFAST CREEK",Café,Fast Food Restaurant,Pub,Turkish Restaurant,American Restaurant,Comedy Club,Thai Restaurant,Cricket Ground,Sushi Restaurant,Recording Studio
1,"ALDERLEY, ENOGGERA, GAYTHORNE, GRANGE, NEWMARK...",Supermarket,Grocery Store,Train Station,Thrift / Vintage Store,Thai Restaurant,Pizza Place,Deli / Bodega,Electronics Store,Dumpling Restaurant,Discount Store
2,"ALGESTER, PARKINSON",Athletics & Sports,Yoga Studio,Dim Sum Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store,Diner
3,"ALTANDI, BANOON, MACGREGOR, ROBERTSON, SUNNYBA...",Korean Restaurant,Grocery Store,Train Station,Athletics & Sports,Martial Arts School,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant
4,"ANNERLEY, FAIRFIELD, FAIRFIELD GARDENS, THOMPS...",Café,Pharmacy,Supermarket,Shopping Mall,Comic Shop,Coffee Shop,Thai Restaurant,Sandwich Place,Discount Store,Fast Food Restaurant
5,"ARCHERFIELD, COOPERS PLAINS",Gas Station,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dessert Shop
6,"ASCOT, BRISBANE AIRPORT, DOOMBEN, HAMILTON, HA...",Pizza Place,Grocery Store,Bakery,Convenience Store,Café,Dessert Shop,Pier,Liquor Store,Bus Station,Burger Joint
7,"ASHGROVE, ASHGROVE EAST, ASHGROVE WEST, DORRIN...",Gym / Fitness Center,Indian Restaurant,Chinese Restaurant,Home Service,Ice Cream Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store
8,"ASPLEY, BOONDALL, CARSELDINE, GEEBUNG, ZILLMERE",Grocery Store,Liquor Store,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store,Diner
9,"AUCHENFLOWER, MOUNT COOT-THA, STUARTHOLME, TOO...",Trail,Yoga Studio,Dim Sum Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store,Diner


In [33]:
df = neighborhoods_venues_sorted

## Display the new dataframe shape. (indicates not all postcodes have FourSquare data as only 64 entries instead of 75)

In [34]:
df.shape

(65, 11)

## Filter results to remove postcodes that already have a 'Coffee Shop'

In [35]:
new = df[df['1st Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new2 = new[new['2nd Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new3 = new2[new2['3rd Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new4 = new3[new3['4th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new5 = new4[new4['5th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new6 = new5[new5['6th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new7 = new6[new6['7th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new8 = new7[new7['8th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new9 = new8[new8['9th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]
new10 = new9[new9['10th Most Common Venue'].apply(lambda x:x not in ['Coffee Shop'])]

## New dataframe has 51 remaining postcodes

In [36]:
new10.shape

(52, 11)

In [37]:
df = new10

## Filter results to remove postcodes that already have a 'Cafe'

In [38]:
new = df[df['1st Most Common Venue'].apply(lambda x:x not in ['Café'])]
new2 = new[new['2nd Most Common Venue'].apply(lambda x:x not in ['Café'])]
new3 = new2[new2['3rd Most Common Venue'].apply(lambda x:x not in ['Café'])]
new4 = new3[new3['4th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new5 = new4[new4['5th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new6 = new5[new5['6th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new7 = new6[new6['7th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new8 = new7[new7['8th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new9 = new8[new8['9th Most Common Venue'].apply(lambda x:x not in ['Café'])]
new10 = new9[new9['10th Most Common Venue'].apply(lambda x:x not in ['Café'])]

## New dataframe has 35 remaining postcodes

In [39]:
new10.shape

(38, 11)

In [40]:
df = new10

## Filter results to remove postcodes that already have a 'Bakery' or 'Diner'

In [41]:
new = df[df['1st Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new2 = new[new['2nd Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new3 = new2[new2['3rd Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new4 = new3[new3['4th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new5 = new4[new4['5th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new6 = new5[new5['6th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new7 = new6[new6['7th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new8 = new7[new7['8th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new9 = new8[new8['9th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]
new10 = new9[new9['10th Most Common Venue'].apply(lambda x:x not in ['Bakery', 'Diner'])]

## New dataframe has 11 remaining postcodes

In [42]:
new10.shape

(11, 11)

In [43]:
df = new10

## Filter results to remove postcodes that already have a 'Grocery Store'

In [44]:
new = df[df['1st Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new2 = new[new['2nd Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new3 = new2[new2['3rd Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new4 = new3[new3['4th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new5 = new4[new4['5th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new6 = new5[new5['6th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new7 = new6[new6['7th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new8 = new7[new7['8th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new9 = new8[new8['9th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]
new10 = new9[new9['10th Most Common Venue'].apply(lambda x:x not in ['Grocery Store'])]

## New dataframe has 7 remaining postcodes

In [45]:
new10.shape

(9, 11)

In [46]:
new10.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"ASHGROVE, ASHGROVE EAST, ASHGROVE WEST, DORRIN...",Gym / Fitness Center,Indian Restaurant,Chinese Restaurant,Home Service,Ice Cream Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store
11,"BALLYMORE, ITHACA, KELVIN GROVE, NORMANBY, RED...",Fast Food Restaurant,Thai Restaurant,Indian Restaurant,Convenience Store,Wine Shop,Italian Restaurant,Salad Place,Pharmacy,Lebanese Restaurant,Beer Store
15,"BAROONA, MILTON, MILTON BC, PADDINGTON, ROSALIE",Italian Restaurant,Vietnamese Restaurant,Gourmet Shop,Japanese Restaurant,Restaurant,Asian Restaurant,Thai Restaurant,Burger Joint,Indie Movie Theater,Indian Restaurant
16,"BELMONT, BELMONT HEIGHTS",Gun Range,Bowling Alley,Athletics & Sports,Dim Sum Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store
26,"CALAMVALE, DREWVALE, STRETTON",Pet Store,Neighborhood,Convenience Store,Business Service,Yoga Studio,Dim Sum Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store
30,"CHELMER, INDOOROOPILLY, INDOOROOPILLY CENTRE, ...",Music Store,Train Station,Multiplex,Indonesian Restaurant,Thai Restaurant,Yoga Studio,Dim Sum Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant
42,"FITZGIBBON, TAIGUM",Train Station,Golf Course,Thai Restaurant,Plaza,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store
59,"SEVENTEEN MILE ROCKS, SINNAMON PARK",Bus Stop,Thai Restaurant,Lake,Playground,Restaurant,Construction & Landscaping,Convenience Store,Cricket Ground,Deli / Bodega,Concert Hall
61,THE GAP,Bus Stop,Trail,Australian Restaurant,Yoga Studio,Dim Sum Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant


## Join Dataframe to get the Long & Lat coordinates

In [47]:
df3 = merged[merged.Neighbourhood.isin(list(new10.Neighbourhood))]

In [48]:
df3

,Post Code,Borough,Neighbourhood,Longitude,Latitude
16,4018,Brisbane - North,"FITZGIBBON, TAIGUM",153.031804,-27.349074
25,4059,Brisbane Inner City,"BALLYMORE, ITHACA, KELVIN GROVE, NORMANBY, RED...",153.008893,-27.452067
26,4060,Brisbane Inner City,"ASHGROVE, ASHGROVE EAST, ASHGROVE WEST, DORRIN...",152.985176,-27.445107
27,4061,Brisbane Inner City,THE GAP,152.944258,-27.450850
28,4064,Brisbane Inner City,"BAROONA, MILTON, MILTON BC, PADDINGTON, ROSALIE",152.999039,-27.464676
32,4068,Brisbane Inner City,"CHELMER, INDOOROOPILLY, INDOOROOPILLY CENTRE, ...",152.976926,-27.506907
33,4073,Brisbane - West,"SEVENTEEN MILE ROCKS, SINNAMON PARK",152.951385,-27.549179
47,4116,Brisbane - South,"CALAMVALE, DREWVALE, STRETTON",153.050793,-27.633988
53,4153,Brisbane - South,"BELMONT, BELMONT HEIGHTS",153.131429,-27.510035


## Display 7 remaining postcodes on the Brisbane map

In [50]:
# create map of Brisbane using latitude and longitude values
map_Brisbane = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186ca',
        fill_opacity=0.7,
        ).add_to(map_Brisbane)  
    
map_Brisbane